In [ ]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 17.4 MB/s eta 0:00:00


In [ ]:
!playwright install


161.3 MiB [] 0% 10.7s161.3 MiB [] 0% 5.8s161.3 MiB [] 0% 4.3s161.3 MiB [] 1% 4.1s161.3 MiB [] 1% 3.7s161.3 MiB [] 2% 3.8s161.3 MiB [] 2% 3.6s161.3 MiB [] 3% 3.6s161.3 MiB [] 3% 3.4s161.3 MiB [] 4% 3.3s161.3 MiB [] 4% 3.2s161.3 MiB [] 5% 3.0s161.3 MiB [] 6% 2.9s161.3 MiB [] 6% 3.2s161.3 MiB [] 7% 3.3s161.3 MiB [] 7% 3.1s161.3 MiB [] 8% 3.1s161.3 MiB [] 8% 3.0s161.3 MiB [] 9% 3.0s161.3 MiB [] 9% 3.1s161.3 MiB [] 10% 3.1s161.3 MiB [] 11% 3.1s161.3 MiB [] 11% 3.0s161.3 MiB [] 12% 3.0s161.3 MiB [] 13% 2.9s161.3 MiB [] 13% 2.8s161.3 MiB [] 14% 2.9s161.3 MiB [] 15% 2.8s161.3 MiB [] 16% 2.8s161.3 MiB [] 17% 2.7s161.3 MiB [] 18% 2.6s161.3 MiB [] 19% 2.5s161.3 MiB [] 20% 2.4s161.3 MiB [] 21% 2.4s161.3 MiB [] 22% 2.3s161.3 MiB [] 23% 2.2s161.3 MiB [] 23% 2.3s161.3 MiB [] 24% 2.3s161.3 MiB [] 24% 2.2s161.3 MiB [] 25% 2.2s161.3 MiB [] 26% 2.1s161.3 MiB [] 27% 2.1s161.3 MiB [] 28% 2.0s161.3 MiB [] 29% 2.0s161.3 MiB [] 29% 2.1s161.3 MiB [] 30% 2.0s161.3 MiB [] 31% 2.0s161.3 MiB [] 32% 1.9s161.3 MiB [

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json
from bs4 import BeautifulSoup
import os

# Jupyter Notebook에서 실행 가능하도록 설정
nest_asyncio.apply()

async def fetch_jnu_scholarship_notices():
    base_url = "https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=list&cate=8&page={}"
    detail_base_url = "https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=view&page=1&key={}&cate=8"
    notices = []
    page_numbers = range(1, 2)  # 5개만 테스트하기 위해 한 페이지만 크롤링

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for page_num in page_numbers:
            url = base_url.format(page_num)
            await page.goto(url)
            soup = BeautifulSoup(await page.content(), "html.parser")

            # ✅ 공지사항 제목 및 key 값 추출
            notice_items = soup.select("td.title a")
            notice_dates = soup.select("td.under:nth-child(3)")  # 날짜 정보가 있는 td

            if not notice_items:
                break

            for item, date_tag in zip(notice_items[:5], notice_dates[:5]):  # 5개만 가져오기
                title = item.text.strip()
                date_str = date_tag.text.strip()
                detail_href = item["href"]

                # ✅ key 값 추출
                if "key=" in detail_href:
                    key = detail_href.split("key=")[-1].split("&")[0]
                else:
                    continue  # key 값이 없으면 무시

                # ✅ 최종 본문 URL 생성
                final_url = detail_base_url.format(key)

                # ✅ 본문 페이지 방문
                await page.goto(final_url)
                final_soup = BeautifulSoup(await page.content(), "html.parser")

                # 🔥 본문 내용 가져오기 (`div.con` 내부)
                content_div = final_soup.find("div", class_="con")
                content = content_div.get_text(strip=True) if content_div else "본문 없음"

                # "장학" 포함된 공지만 저장
                if "장학" in title or "장학" in content:
                    notices.append({"title": title, "url": final_url, "date": date_str, "content": content})

        await browser.close()

    return notices

# 크롤링 실행
notices_data = asyncio.run(fetch_jnu_scholarship_notices())
print(f"총 {len(notices_data)}개의 장학 공지사항을 크롤링했습니다.\n")

# ✅ 크롤링된 데이터 출력 (최대 5개 미리보기)
for i, notice in enumerate(notices_data, 1):
    content_preview = notice['content'][:300]  # 본문 앞 300자만 출력
    print(f"🔹 [{i}] {notice['title']} ({notice['date']})")
    print(f"   👉 {notice['url']}")
    print(f"   📄 {content_preview}...\n")


총 5개의 장학 공지사항을 크롤링했습니다.

🔹 [1] [장학안내] 2025학년도 주거안정장학금 신청 안내 (학생과)
   👉 https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=view&page=1&key=65084&cate=8
   📄 <2025학년도 주거안정장학금 신청 안내>가.신청대상:재학생*,재입학생,신입생,편입생*재학생은 직전학기12학점 이상 이수자로C학점(70/100점)이상을 획득한 자나.지원대상:부모 주소지가 소속 대학 기준으로 통학이 어려운원거리 지역*에 해당하는미혼기초·차상위(한국장학재단 당해 학기 학자금 지원 구간 산정)대학생※단,청년 주거비 지원 사업(청년 주거급여 분리 지급,청년 월세 특별지원 등)수혜자는 제외*｢대도시권 광역교통 관리에 관한 특별법｣등을 바탕으로 원거리 통학 기준 마련,한국장학재단 홈페이지에서 지원불가 지역 확인 가능다.지원내...

🔹 [2] [장학안내] 2025학년도 주거안정장학금 신청 안내 (학생과)
   👉 https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=view&page=1&key=65083&cate=8
   📄 <2025학년도 주거안정장학금 신청 안내>가.신청대상:재학생*,재입학생,신입생,편입생*재학생은 직전학기12학점 이상 이수자로C학점(70/100점)이상을 획득한 자나.지원대상:부모 주소지가 소속 대학 기준으로 통학이 어려운원거리 지역*에 해당하는미혼기초·차상위대학생※단,청년 주거비 지원 사업(청년 주거급여 분리 지급,청년 월세 특별지원 등)수혜자는 제외*｢대도시권 광역교통 관리에 관한 특별법｣등을 바탕으로 원거리 통학 기준 마련,한국장학재단 홈페이지에서 지원불가 지역 확인 가능다.지원내용:월 최대20만원,연간 최대240만원*방학 중은...

🔹 [3] [장학안내] [세아해암학술장학재단] 제4기 커리어챌린지 장학생 모집 안내 (학생과)


In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json
from bs4 import BeautifulSoup
import os

# Jupyter Notebook에서 실행 가능하도록 설정
nest_asyncio.apply()

async def fetch_jnu_scholarship_notices():
    base_url = "https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=list&cate=8&page={}"
    detail_base_url = "https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=view&page=1&key={}&cate=8"
    notices = []
    page_numbers = range(1, 21)  # 200개 크롤링 (1~20 페이지, 페이지당 약 10개)

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for page_num in page_numbers:
            url = base_url.format(page_num)
            await page.goto(url)
            soup = BeautifulSoup(await page.content(), "html.parser")

            # ✅ 공지사항 제목 및 key 값 추출
            notice_items = soup.select("td.title a")
            notice_dates = soup.select("td.under:nth-child(3)")  # 날짜 정보가 있는 td

            if not notice_items:
                break

            for item, date_tag in zip(notice_items, notice_dates):
                title = item.text.strip()
                date_str = date_tag.text.strip()
                detail_href = item["href"]

                # ✅ key 값 추출
                if "key=" in detail_href:
                    key = detail_href.split("key=")[-1].split("&")[0]
                else:
                    continue  # key 값이 없으면 무시

                # ✅ 최종 본문 URL 생성
                final_url = detail_base_url.format(key)

                # ✅ 본문 페이지 방문
                await page.goto(final_url)
                final_soup = BeautifulSoup(await page.content(), "html.parser")

                # 🔥 본문 내용 가져오기 (`div.con` 내부)
                content_div = final_soup.find("div", class_="con")
                content = content_div.get_text(strip=True) if content_div else "본문 없음"

                # "장학" 포함된 공지만 저장
                if "장학" in title or "장학" in content:
                    notices.append({"title": title, "url": final_url, "date": date_str, "content": content})

        await browser.close()

    # JSON 파일 저장
    save_path = "jnu_scholarship_200.json"
    if os.path.exists(save_path):
        os.remove(save_path)  # 기존 파일 삭제
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(notices, f, ensure_ascii=False, indent=4)

    return notices

# 크롤링 실행
notices_data = asyncio.run(fetch_jnu_scholarship_notices())
print(f"총 {len(notices_data)}개의 장학 공지사항을 크롤링했습니다.\n")

# ✅ 크롤링된 데이터 출력 (최대 5개 미리보기)
for i, notice in enumerate(notices_data[:5], 1):
    content_preview = notice['content'][:300]  # 본문 앞 300자만 출력
    print(f"🔹 [{i}] {notice['title']} ({notice['date']})")
    print(f"   👉 {notice['url']}")
    print(f"   📄 {content_preview}...\n")


총 400개의 장학 공지사항을 크롤링했습니다.

🔹 [1] [장학안내] 2025학년도 주거안정장학금 신청 안내 (학생과)
   👉 https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=view&page=1&key=65084&cate=8
   📄 <2025학년도 주거안정장학금 신청 안내>가.신청대상:재학생*,재입학생,신입생,편입생*재학생은 직전학기12학점 이상 이수자로C학점(70/100점)이상을 획득한 자나.지원대상:부모 주소지가 소속 대학 기준으로 통학이 어려운원거리 지역*에 해당하는미혼기초·차상위(한국장학재단 당해 학기 학자금 지원 구간 산정)대학생※단,청년 주거비 지원 사업(청년 주거급여 분리 지급,청년 월세 특별지원 등)수혜자는 제외*｢대도시권 광역교통 관리에 관한 특별법｣등을 바탕으로 원거리 통학 기준 마련,한국장학재단 홈페이지에서 지원불가 지역 확인 가능다.지원내...

🔹 [2] [장학안내] 2025학년도 주거안정장학금 신청 안내 (학생과)
   👉 https://www.jnu.ac.kr/WebApp/web/HOM/COM/Board/board.aspx?boardID=5&bbsMode=view&page=1&key=65083&cate=8
   📄 <2025학년도 주거안정장학금 신청 안내>가.신청대상:재학생*,재입학생,신입생,편입생*재학생은 직전학기12학점 이상 이수자로C학점(70/100점)이상을 획득한 자나.지원대상:부모 주소지가 소속 대학 기준으로 통학이 어려운원거리 지역*에 해당하는미혼기초·차상위대학생※단,청년 주거비 지원 사업(청년 주거급여 분리 지급,청년 월세 특별지원 등)수혜자는 제외*｢대도시권 광역교통 관리에 관한 특별법｣등을 바탕으로 원거리 통학 기준 마련,한국장학재단 홈페이지에서 지원불가 지역 확인 가능다.지원내용:월 최대20만원,연간 최대240만원*방학 중은...

🔹 [3] [장학안내] [세아해암학술장학재단] 제4기 커리어챌린지 장학생 모집 안내 (학생과

In [ ]:
import os
from google.colab import files

save_path = "/content/jnu_scholarship_200.json"

# ✅ 기존 파일 삭제 후 다시 저장
if os.path.exists(save_path):
    os.remove(save_path)
    print("🗑️ 기존 JSON 파일 삭제 완료")

# ✅ 최신 JSON 데이터 저장
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(notices_data, f, ensure_ascii=False, indent=4)

# 파일 다운로드
files.download(save_path)

print(f"✅ JSON 파일이 저장되었고 다운로드할 수 있습니다: {save_path}")


🗑️ 기존 JSON 파일 삭제 완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ JSON 파일이 저장되었고 다운로드할 수 있습니다: /content/jnu_scholarship_200.json
